# read in, clean up, data
### inflation, EquityReturns, Unemployment, 
### consumer sentiment, home buyers index
### get into linear form for dataframe, database

In [35]:
#dependencies
import sqlite3
import pandas as pd
import datetime

In [36]:
rawdatapath='EconomicDataRaw/'

### read in CPI, convert to inflation, setup monthly dataframe

In [37]:
CPI_df=pd.read_excel(rawdatapath+'CPI_bls.xlsx',skiprows=10,header=1)
CPI_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
0,1913,29.400,29.300,29.300,29.400,29.200,29.300,29.600,29.800,29.900,30.100,30.200,30.100,NaN,NaN
1,1914,30.100,29.800,29.700,29.400,29.600,29.800,30.100,30.500,30.600,30.400,30.500,30.400,NaN,NaN
2,1915,30.300,30.100,29.800,30.100,30.200,30.300,30.300,30.300,30.400,30.700,30.900,31.000,NaN,NaN
3,1916,31.300,31.300,31.600,31.900,32.000,32.400,32.400,32.800,33.400,33.800,34.400,34.600,NaN,NaN
4,1917,35.000,35.800,36.000,37.600,38.400,38.800,38.400,39.000,39.700,40.400,40.500,41.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2017,727.439,729.727,730.320,732.486,733.110,733.775,733.269,735.466,739.359,738.893,738.912,738.477,731.143,737.396
105,2018,742.499,745.866,747.554,750.524,753.647,754.848,754.898,755.317,756.194,757.532,754.992,752.584,749.156,755.253
106,2019,754.017,757.204,761.477,765.507,767.138,767.291,768.571,768.533,769.136,770.895,770.482,769.781,762.106,769.566
107,2020,772.768,774.886,773.199,768.029,768.044,772.245,776.152,778.599,779.684,780.008,779.531,780.263,771.529,779.040


In [38]:
# create monthly dates to use in dataframe
start=datetime.datetime(2000,1,1)
end=datetime.datetime(2021,1,31)
months=pd.date_range(start,end,freq="BM")


In [39]:
#create dataframe
df_econ=pd.DataFrame({'Date':months})

In [40]:
#turn cpi from year/month table into monthly list
CPI=[]
for date in df_econ['Date']:
    cpival=CPI_df.loc[CPI_df["Year"]==date.year].iloc[0,[date.month]].values[0]  # values[0] turns 1 element array to a mumber
    CPI.append(cpival)
# convert this to an inflation rate
# divide cpi/prior period cpi, multiply by 12
CPInum=CPI[1:]  
CPIden=CPI[:-1]
inflation=[12*((n/d)-1) for n,d in zip(CPInum,CPIden)]
inflation.insert(0,0)  # had to cut an element of of num, den - get this back to proper spacing and time period
df_econ['inflation']=inflation
df_econ.set_index('Date',inplace=True)
df_econ.head(7)
    
    

    

,inflation
Date,
2000-01-31,0.000000
2000-02-29,0.068802
2000-03-31,0.096717
2000-04-28,0.009360
2000-05-31,0.009353
2000-06-30,0.067757
2000-07-31,0.023233


## equity returns (S&P)

In [41]:
SandP_df=pd.read_csv(rawdatapath+'SandP_Yahoo.csv')
# only column I want to keep is Close; drop everything else
SandP_df.drop(columns=['Open','High','Low','Adj Close','Volume'],inplace=True)
SandP_df.head()

,Date,Close
0,1985-01-01,179.630005
1,1985-02-01,181.179993
2,1985-03-01,180.660004
3,1985-04-01,179.830002
4,1985-05-01,189.550003


In [42]:
SandP_df.dtypes


Date      object
Close    float64
dtype: object

In [43]:
# turn "Date" column into date-time object
dto=[]                      # list to hold date time object
for date in SandP_df['Date']:
    dto.append(datetime.datetime.strptime(date,'%Y-%m-%d'))
SandP_df['DTdate']=dto
SandP_df.set_index('DTdate',inplace=True)
SandP_df.drop(columns=['Date'],inplace=True)
SandP_df.head()


,Close
DTdate,
1985-01-01,179.630005
1985-02-01,181.179993
1985-03-01,180.660004
1985-04-01,179.830002
1985-05-01,189.550003


In [48]:
qwell=pd.concat([df_econ,SandP_df],axis=1,join='inner')
qwell

,inflation,Close


In [49]:
SandP_df

,Close
DTdate,
1985-01-01,179.630005
1985-02-01,181.179993
1985-03-01,180.660004
1985-04-01,179.830002
1985-05-01,189.550003
...,...
2020-11-01,3621.629883
2020-12-01,3756.070068
2021-01-01,3714.239990


In [61]:
qwell=SandP_df.iloc[1]
qwell

Close    181.179993
Name: 1985-02-01 00:00:00, dtype: float64

In [64]:
df_econ.to_csv('df_econ.csv')